In [1]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

cmd = cmdgen.CommandGenerator()

In [2]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.3.6.1.2.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

errorIndication: No SNMP response received before timeout
errorStatus: 0
errorIndex: 0
varBinds: []


chexiang
===

- system = 1.3.6.1.2.1.1
- sysName = 1.3.6.1.2.1.1.5
- ifEntry = 1.3.6.1.2.1.2.2.1
- ifDescr = 1.3.6.1.2.1.2.2.1.2
- ifSpeed = 1.3.6.1.2.1.2.2.1.5

In [73]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
cmdGen = cmdgen.CommandGenerator()  # 同步
cmdGen = cmdgen.AsynCommandGenerator()  # 异步

mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
)
mibView = view.MibViewController(mibBuilder)


mib_vars = (
    cmdgen.MibVariable('IF-MIB', 'ifOperStatus'),
    cmdgen.MibVariable('IF-MIB', 'ifSpeed'),
    cmdgen.MibVariable('IF-MIB', 'ifInErrors'),
)


def callback(sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
    print('*'*50)
    print(errorIndication)
    print(errorStatus)
    print(errorIndex)
    print(cbCtx)
    print(varBinds)
    for row in varBinds:
        for oid, val in row:
            print(oid.prettyPrint(), val.prettyPrint())
#             print(oid.getMibSymbol()[-1][0], val.prettyPrint())
            

# def callback(*args, **kw):
#     for a in args:
#         print(a)
#     print(kw)

for mib_var in mib_vars:
    cmdGen.nextCmd(
        cmdgen.CommunityData('huawei@123'),
        cmdgen.UdpTransportTarget(('10.32.33.6', 161)),
        (mib_var,),
#         (cmdgen.MibVariable('IF-MIB', 'ifOperStatus'),),
        (callback, 'abc'),
        True, True
    )

    
# 启动
cmdGen.snmpEngine.transportDispatcher.runDispatcher()


# print(errorIndication)
# print(errorStatus)
# print(errorIndex)
# print('*' * 50)


# # print(varBindTable)
# for row in varBindTable:
#     for oid, val in row:
#         #         print(repr(oid))
#         print(oid.getMibSymbol()[-1][0], val.prettyPrint())


**************************************************
None
0
0
abc
[[ObjectType(ObjectIdentity(ObjectName('1.3.6.1.2.1.2.2.1.8.1')), Integer32('up', subtypeSpec=ConstraintsIntersection(ConstraintsIntersection(ConstraintsIntersection(), ValueRangeConstraint(-2147483648, 2147483647)), SingleValueConstraint(1, 2, 3, 4, 5, 6, 7)), NamedValues(('up', 1), ('down', 2), ('testing', 3), ('unknown', 4), ('dormant', 5), ('notPresent', 6), ('lowerLayerDown', 7))))]]
IF-MIB::ifOperStatus."1" 'up'
**************************************************
None
0
0
abc
[[ObjectType(ObjectIdentity(ObjectName('1.3.6.1.2.1.2.2.1.5.1')), Gauge32(0))]]
IF-MIB::ifSpeed."1" 0
**************************************************
None
0
0
abc
[[ObjectType(ObjectIdentity(ObjectName('1.3.6.1.2.1.2.2.1.14.1')), Counter32(0))]]
IF-MIB::ifInErrors."1" 0


In [69]:
cmdGen.nextCmd??